In [1]:
import random
import openslide
import h5py
import pandas as pd
import numpy as np

from HIPT.HIPT_4K.hipt_4k import HIPT_4K
from HIPT.HIPT_4K.hipt_model_utils import get_vit256, get_vit4k, eval_transforms
from HIPT.HIPT_4K.hipt_heatmap_utils import *

import torch
import os
from tqdm.notebook import tqdm
torch.cuda.get_device_name(0),torch.cuda.get_device_name(1)

('Tesla V100-SXM2-32GB', 'Tesla V100-SXM2-32GB')

In [2]:
df_path='/home/ss4yd/vision_transformer/captioning_vision_transformer/prepared_prelim_data.csv'
df=pd.read_csv(df_path)
df

,pid,tissue_type,notes,svs_path,patch_path,reps_path
0,GTEX-1117F-0726,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
1,GTEX-1117F-1026,Lung,"2 pieces, moderate congestion/moderate to mark...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
2,GTEX-1117F-1126,Liver,"2 pieces, subtotal massive hepatic necrosis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
3,GTEX-1117F-1226,Spleen,"2 pieces, marked congestion/autolysis",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
4,GTEX-1117F-1326,Adipose - Visceral (Omentum),"2 pieces, diffuse mesothelial hyperplasia; ~10...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
...,...,...,...,...,...,...
1402,GTEX-13SLW-0726,Thyroid,"2 pieces, colloid cysts up to ~1.5mm, rep deli...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
1403,GTEX-13SLW-0826,Heart - Atrial Appendage,"2 pieces, no abnormalities",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
1404,GTEX-13SLW-0926,Artery - Aorta,"6 pieces, atherosis up to ~0.5mm",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...
1405,GTEX-13SLW-1026,Heart - Left Ventricle,"2 pieces, moderate ischemic changes/fibrosis, ...",/project/GutIntelligenceLab/ss4yd/gtex_data/ac...,/project/GutIntelligenceLab/ss4yd/gtex_data/pa...,/project/GutIntelligenceLab/ss4yd/gtex_data/hi...


In [4]:
def get_model():
    pretrained_weights256 = '/home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth'
    pretrained_weights4k = '/home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit4k_xs_dino.pth'

    if torch.cuda.is_available():
        device256 = torch.device('cuda:0')
        device4k = torch.device('cuda:1')
    else:
        device256 = torch.device('cpu')
        device4k = torch.device('cpu')


    ### ViT_256 + ViT_4K loaded independently (used for Attention Heatmaps)
    model256 = get_vit256(pretrained_weights=pretrained_weights256, device=device256)
    model4k = get_vit4k(pretrained_weights=pretrained_weights4k, device=device4k)

    ### ViT_256 + ViT_4K loaded into HIPT_4K API
    model = HIPT_4K(pretrained_weights256, pretrained_weights4k, device256, device4k)
    model.eval()
    return model

In [5]:
model=get_model()

Take key teacher in provided checkpoint dict
Pretrained weights found at /home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
# of Patches: 196
Take key teacher in provided checkpoint dict
Pretrained weights found at /home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit4k_xs_dino.pth and loaded with msg: _IncompatibleKeys(missing_keys=[], unexpected_keys=['head.mlp.0.weight', 'head.mlp.0.bias', 'head.mlp.2.weight', 'head.mlp.2.bias', 'head.mlp.4.weight', 'head.mlp.4.bias', 'head.last_layer.weight_g', 'head.last_layer.weight_v'])
Take key teacher in provided checkpoint dict
Pretrained weights found at /home/ss4yd/vision_transformer/HIPT/HIPT_4K/Checkpoints/vit256_small_dino.pth and loaded with msg: _Incompatibl

In [6]:
def get_reps(model, slide, coords, size=4096):
#     print(coords)
    x,y=coords
    region=slide.read_region((x,y),0,(size,size)).convert('RGB')
    x = eval_transforms()(region).unsqueeze(dim=0)
    with torch.no_grad():
        out = model.forward(x)
    return out.cpu()

In [7]:
save_path='/project/GutIntelligenceLab/ss4yd/gtex_data/hipt4kreps/'
os.makedirs(save_path, exist_ok=True)

# patch_dict={key:value for key,value in zip(df['pid'], df['patch_path'])}
for index, row in df.iterrows():
    patch_path=row['patch_path']
    svs_path=row['svs_path']
    pid=row['pid']
    print('\nSVS ID: '+f'{pid}')
    patch_rep_list=[]
    coords = h5py.File(patch_path, 'r')['coords']
    slide=openslide.open_slide(svs_path)
    print('Number of patches: '+f'{len(coords)}')
    for coord in coords:
        patch_rep_list.append(get_reps(model,slide,coord))
    
    tensor=torch.stack(patch_rep_list)
    torch.save(tensor,save_path+ f'{pid}.pt')
    print('Finished saving tensor..')
    print('Progress: '+ f'{(index*100)/len(df)}%')
    break


SVS ID: GTEX-1117F-0726
Number of patches: 21
Finished saving tensor..
Progress: 0.0%


In [8]:
tensor.shape

torch.Size([21, 1, 192])

In [9]:
tensor

tensor([[[ 4.0323,  2.1642, -1.6184,  ..., -2.7638,  0.4952, -0.7379]],

        [[ 2.0881,  2.4777, -0.7175,  ..., -2.7558,  1.4247, -1.6451]],

        [[ 1.4498,  3.1653,  0.3425,  ..., -2.4972,  1.6075, -1.4261]],

        ...,

        [[ 2.8392,  2.3482, -1.6898,  ..., -1.5298, -0.4011, -0.4610]],

        [[ 1.6531,  0.8023, -2.0554,  ...,  0.1655,  1.2741, -0.2307]],

        [[ 3.1640,  1.7448, -0.1243,  ..., -1.0070, -0.4244, -1.2139]]])

In [10]:
torch.load('/project/GutIntelligenceLab/ss4yd/gtex_data/hipt4kreps/GTEX-1117F-1026.pt')

tensor([[[ 1.9866,  0.1871, -1.7661,  ..., -3.1260,  0.8078, -1.6956]],

        [[ 1.3729,  0.7826, -2.4570,  ..., -2.4590,  0.2865, -1.7576]],

        [[ 1.6081,  0.4727, -2.1768,  ..., -2.8607,  1.1561, -1.3885]],

        ...,

        [[ 2.1131,  1.1773, -3.8304,  ..., -2.9065, -1.0534, -1.2344]],

        [[ 1.7868, -0.5002, -3.1622,  ..., -2.7068,  0.6237, -1.1603]],

        [[ 2.0311,  0.7627, -3.5793,  ..., -3.5482, -0.4108, -1.6977]]])